In [28]:
import requests
from datetime import datetime, timedelta, timezone

# === Configuration ===
API_KEY = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6IjdiYjVkMjVlMjhmMDQ0ZmQ5ZGZiYmFiMDI5YmIzMGU3IiwiaCI6Im11cm11cjEyOCJ9"  # Replace with your real KNMI API key
BASE_URL = "https://api.dataplatform.knmi.nl/edr/v1"
COLLECTION = "10-minute-in-situ-meteorological-observations"
station_id = "0-20000-0-06203"

# === Time window: last 30 minutes ===
def valid_datetime_input(timepoint: datetime):
    return timepoint.isoformat(timespec="seconds").replace("+00:00", "Z")


now = datetime.now(timezone.utc)
start = now - timedelta(minutes=30)
time_range = f"{valid_datetime_input(start)}/{valid_datetime_input(now)}"
now = valid_datetime_input(now)

print(time_range)
# === API request ===
params = {
    "datetime": f"P1D/{time_range}",
    "parameter-name": "air_pressure_at_sea_level",
    "crs": "EPSG:4326"
}

headers = {
    "Authorization": API_KEY
}


# List available collections to verify the collection name
response = requests.get(
    f"{BASE_URL}/collections/{COLLECTION}/locations/{station_id}",
    headers=headers,
    params=params,
)
response.raise_for_status()
data = response.json()

data


2025-07-06T13:46:12Z/2025-07-06T14:16:12Z


HTTPError: 400 Client Error: Bad Request for url: https://api.dataplatform.knmi.nl/edr/v1/collections/10-minute-in-situ-meteorological-observations/locations/0-20000-0-06203?datetime=P1D%2F2025-07-06T13%3A46%3A12Z%2F2025-07-06T14%3A16%3A12Z&parameter-name=air_pressure_at_sea_level&crs=EPSG%3A4326

In [20]:
features = data.get("features", [])
if not features:
    print("No data found.")
else:
    print(f"Found {len(features)} measurements for air pressure.\n")
    for f in features:
        station = f.get("id")
        coords = f.get("geometry", {}).get("coordinates", [])
        props = f.get("properties", {})
        time = props.get("datetime")

        pressure_obj = props.get("parameter", {}).get("air_pressure_at_sea_level")
        pressure = pressure_obj.get("value") if pressure_obj else None

        print(f"Station: {station}")
        print(f"  Time: {time}")
        print(f"  Pressure: {pressure} hPa")
        print(f"  Location (lon, lat): {coords[:2]}")
        print("-" * 30)


Found 64 measurements for air pressure.

Station: 0-20000-0-06203
  Time: None
  Pressure: None hPa
  Location (lon, lat): [3.3417, 52.36]
------------------------------
Station: 0-20000-0-06204
  Time: None
  Pressure: None hPa
  Location (lon, lat): [3.6278, 53.2694]
------------------------------
Station: 0-20000-0-06207
  Time: None
  Pressure: None hPa
  Location (lon, lat): [4.9603, 53.6144]
------------------------------
Station: 0-20000-0-06208
  Time: None
  Pressure: None hPa
  Location (lon, lat): [5.9417, 53.4917]
------------------------------
Station: 0-20000-0-06209
  Time: None
  Pressure: None hPa
  Location (lon, lat): [4.5175, 52.4636]
------------------------------
Station: 0-20000-0-06215
  Time: None
  Pressure: None hPa
  Location (lon, lat): [4.4364, 52.1397]
------------------------------
Station: 0-20000-0-06225
  Time: None
  Pressure: None hPa
  Location (lon, lat): [4.555, 52.4622]
------------------------------
Station: 0-20000-0-06229
  Time: None
  Press

In [21]:
import json
print(json.dumps(data["features"][0], indent=2))

{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      3.3417,
      52.36
    ]
  },
  "properties": {
    "name": "P11-B",
    "wmoId": "06203",
    "height_above_mean_sea_level": 41.84
  },
  "id": "0-20000-0-06203"
}
